In [62]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Conv2D
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.initializers import RandomNormal

In [3]:
mixedwm = np.load('/Users/roopvankayalapati/Documents/Projects/Wafer defect detection/MixedWM38.npz')

In [4]:
mixedwm.files

['arr_0', 'arr_1']

In [5]:
mixedwm['arr_1']

array([[1, 0, 1, ..., 0, 1, 0],
       [1, 0, 1, ..., 0, 1, 0],
       [1, 0, 1, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int32)

[‘arr_0’]: Defect data of mixed-type wafer map, 0 means blank spot, 1 represents normal die that passed the electrical test, and 2 represents broken die that failed the electrical test. The data shape is 52×52.

[‘arr_1’]: Mixed-type wafer map defect label, using one-hot encoding, a total of 8 dimensions, corresponding to the 8 basic types of wafer map defects (single defect).

In [6]:
mixedwm['arr_0'].shape


(38015, 52, 52)

In [7]:
mixedwm['arr_1'].shape

(38015, 8)

In [8]:
mixedwm['arr_0']

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [9]:
X_train, X_test, y_train, y_test = train_test_split(mixedwm['arr_0'], mixedwm['arr_1'], test_size=0.2, random_state=1)

In [25]:
print(X_train.shape)
print(y_train.shape)

(30412, 52, 52)
(30412, 8)


In [27]:
X_train.shape[1:]

(52, 52)

In [26]:
print(X_test.shape)
print(y_test.shape)

(7603, 52, 52)
(7603, 8)


Start Building model

In [37]:
def tf_repeat_2d(a, repeats):
    assert len(a.get_shape()) == 2
    a = tf.expand_dims(a, 0)
    a = tf.tile(a, [repeats, 1, 1])
    return a

In [39]:
def tf_batch_map_offsets(input, offsets, order = 1):
    #input: tf.Tensor.shape = (b, s, s)
    #offsets: tf.Tensor.Shape = (b, s, s, 2)
    #returns: tf.Tensor.shape = (b, s, s)
    
    input_shape = tf.shape(input)
    batch_size = input_shape[0]
    input_size = input_shape[1]
    
    offsets = tf.reshape(offsets, (batch_size, -1, 2))
    grid = tf.meshgrid(tf.range(input_size), tf.range(input_size), indexing = 'ij')
    grid = tf.stack(grid, axis = -1)
    grid = tf.cast(grid, 'float32')
    grid = tf.reshape(grid, (-1, 2))
    grid = tf_repeat_2d(grid, batch_size)
    coords = grid + offsets
    
    mapped_vals = tf_batch_map_coordinates(input, coords)
    
    return mapped_vals

In [74]:
class ConvOffset2D_train(Conv2D):
    '''
    Convolutional layer responsible for learning the 2D offsets and output the deformed
    feature map using bilinear interpolation
    '''
    def __init__(self, filters, init_normal_stddev = 0.01, **kwargs):
        '''
        filters: int
        Number of channel of the input feature map
        init_normal_stddev: float
        Normal kernel initialization
        **kwargs:
        pass to superclass.
        '''
        self.filters = filters
        super(ConvOffset2D_train, self).__init__(self.filters * 2, (3, 3), padding = 'same',use_bias = False, 
                                                 kernel_initializer = RandomNormal(0, init_normal_stddev))
        
    def call(self, x):
        '''
        return the deformed featureed map
        '''
        x_shape = x.get_shape()
        #offsets = super(ConvOffset2D_train, self).call(x)
        offsets = self.conv2d(x)

        # offsets: (b*c, h, w, 2)
        offsets = self._to_bc_h_w_2(offsets, x_shape)
        # x: (b*c, h, w)
        x = self._to_bc_h_w(x, x_shape)
        # X_offset: (b*c, h, w)
        x_offset = tf_batch_map_offsets(x, offsets)
        # x_offset: (b, h, w, c)
        x_offset = self._to_b_h_w_c(x_offset, x_shape)

        return x_offset

    def compute_output_shape(self, input_shape):
        '''
        Output shape is the same as input shape
        Becauase, this layer only does the deformation part
        '''
        return input_shape
    
    @staticmethod
    def _to_bc_h_w_2(x, x_shape):
        '''
        (b, h, w, 2c)->(bc, h, w, 2)
        '''
        x = tf.transpose(x, [0, 3, 1, 2])
        x = tf.reshape(x, (-1, int(x_shape[1]), int(x_shape[2]), 2))
        return x

    @staticmethod
    def _to_bc_h_w(x, x_shape):
        '''
        (b, h, w, c)->(bc, h, w)
        '''
        x = tf.transpose(x, [0, 3, 1, 2])
        x = tf.reshape(x, (-1, int(x_shape[1]), int(x_shape[2])))
        return x

    @staticmethod
    def _to_b_h_w_c(x, x_shape):
        '''
        (b*c, h, w)->(b, h, w, c)
        '''
        x = tf.reshape(x, (-1, int(x_shape[3]), int(x_shape[1]), int(x_shape[2])))
        x = tf.transpose(x, [0, 2, 3, 1])
        return x

In [96]:
class Train_mix():
    def __init__(self,datapath = '/Users/roopvankayalapati/Documents/Projects/Wafer defect detection/MixedWM38.npz',
                 lr=0.0001,fullnet_num=128,conv_num=32,deconv_size=(3,3)):
        self.datapath=datapath
        self.lr = lr
        self.fullnet_num = fullnet_num
        self.conv_num = conv_num
        self.deconv_size = deconv_size
    
    def auc1(self,inputs,pre):
        inputs_T=inputs.T
        pre_T=pre.T
        #for i in range(len(inputs_T)):
        acc=np.mean(np.equal(inputs_T,pre_T).astype(np.float64),axis=1)
        return acc

    def lrelu(self,x, leak=0.2, name="lrelu"):
        with tf.variable_scope(name):
            f1 = 0.5 * (1 + leak)
            f2 = 0.5 * (1 - leak)
            return f1 * x + f2 * abs(x)

    def acc_myself(self,y_true, y_pre):
        y_pre = tf.round(y_pre)
        r = tf.equal(y_true, y_pre)
        r = tf.cast(r, tf.float32)
        r = tf.reduce_sum(r, axis=1)
        d = tf.zeros_like(r, dtype=tf.float32) + 8
        c = tf.equal(r, d)
        c = tf.cast(c, tf.float32)

        return tf.divide(tf.reduce_sum(c), tf.cast(tf.size(c), tf.float32))

    
    def build_model(self,inputs_shape,classes=8,trainable=True):
        bn_axis=3

        inputs=Input(shape=inputs_shape)
        #x = ConvOffset2D_train(1, name='conv_1_offset')(inputs)
        x = Conv2D(self.conv_num, (3, 3), strides=(2, 2), padding='same', name='conv_1', trainable=trainable)(inputs)
        x = BatchNormalization(axis=bn_axis, name='batch_normalization_1')(x)
        x = Activation('relu', name='activation_1')(x)

        # Conv_2 layer
        #x = ConvOffset2D_train(32, name='conv_2_offset')(x)
        x = Conv2D(self.conv_num*2, (3, 3), strides=(2, 2), padding='same', name='conv_2', trainable=trainable)(x)
        x = BatchNormalization(axis=bn_axis, name='batch_normalization_2')(x)
        x = Activation('relu', name='activation_2')(x)

        # Conv_3 layer
        #x = ConvOffset2D_train(64, name='conv_3_offset')(x)
        x = Conv2D(self.conv_num*4, (3, 3), strides=(2, 2), padding='same', name='conv_3', trainable=trainable)(x)
        x = BatchNormalization(axis=bn_axis, name='batch_normalization_3')(x)
        x = Activation('relu', name='activation_3')(x)

        # Conv_4 layer
        #x = ConvOffset2D_train(128, name='conv_4_offset')(x)
        x = Conv2D(self.conv_num*8, (3, 3), padding='same', name='conv_4', trainable=trainable)(x)
        x = BatchNormalization(axis=bn_axis, name='batch_normalization_4')(x)
        x = Activation('relu', name='activation_4')(x)

        # Conv_5 layer
        #x = ConvOffset2D_train(256, name='conv_5_offset')(x)
        x = Conv2D(self.conv_num*4, (3, 3), strides=(2, 2), padding='same', name='conv_5', trainable=trainable)(x)
        x = BatchNormalization(axis=bn_axis, name='batch_normalization_5')(x)
        x = Activation('relu', name='activation_5')(x)

        # Pooling layer
        x = GlobalAveragePooling2D()(x)

        # fc layer
        outputs = Dense(classes, activation='sigmoid', name='fc', trainable=trainable)(x)

        return inputs, outputs
    
    def start_train(self):
        
        data=np.load(self.datapath)
        trainx=data["arr_0"]
        trainy = data["arr_1"]


        trainx=np.expand_dims(trainx,axis=-1)

        data_shape=trainx.shape[1:]

        inputs, outputs = self.build_model(data_shape, classes=trainy.shape[-1], trainable=True)

        model = Model(inputs=inputs, outputs=outputs)
        model.summary()

        loss = keras.losses.binary_crossentropy
        #loss = keras.losses.categorical_crossentropy
        optimizer = keras.optimizers.legacy.SGD(learning_rate=self.lr, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss=loss, optimizer=optimizer, metrics=[self.acc_myself])
        
        x_train, x_test, y_train, y_test = train_test_split(trainx, trainy, test_size=0.2, random_state=10)
        
        history = model.fit(x_train, y_train, epochs = 50, batch_size = 32, validation_data = (x_test, y_test))
        return history
    



In [97]:
trainer = Train_mix(datapath = '/Users/roopvankayalapati/Documents/Projects/Wafer defect detection/MixedWM38.npz',
                    lr = 0.001, fullnet_num = 128, conv_num = 32, deconv_size = (3,3))

In [ ]:
trainer.start_train()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 52, 52, 1)]       0         
                                                                 
 conv_1 (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 batch_normalization_1 (Bat  (None, 26, 26, 32)        128       
 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv_2 (Conv2D)             (None, 13, 13, 64)        18496     
                                                                 
 batch_normalization_2 (Bat  (None, 13, 13, 64)        256       
 chNormalization)                                          